# XGBoost - Multilabel Classification

This notebook optimises hyperparameters for LightGBM using Optuna.

[Source kernel](https://www.kaggle.com/pavelvpster/moa-lgb-optuna)

In [21]:
%load_ext autoreload
%aimport numpy, matplotlib, pandas, category_encoders, sklearn, xgboost
%autoreload 2

import lightgbm as lgb
import matplotlib.pyplot as plt
import numpy as np
import optuna
import pandas as pd
%matplotlib inline

from datetime import datetime
from sklearn.metrics import log_loss
from sklearn.model_selection import StratifiedKFold
from sklearn.multioutput import MultiOutputClassifier
from sklearn.pipeline import Pipeline
from time import time

from src.data.make_dataset import get_base_datasets

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [22]:
plt.style.use("../style.mplstyle")
SEED = 42
NFOLDS = 5
np.random.seed(SEED)

## Load Data

- `df`: feature data.
 - 772 gene expression features and 100 cell viability features. Also:
   - `cp_type`: indicates whether the experiment is a treatment (contains drug) or a control (contains no drug - probably DMSO, which has negligible biological effects).
   - `cp_dose`: the dose level used in the experiment. Generally, higher dose leads to stronger effect.
   - `cp_time`: time elapsed between adding the drug and taking the measurement.
   - `flag`: specifies if row is training (n=23,814) or test (n=3,982) data.
 - One row = one drug at a specific dose (high/low) and time point (24/48/72 hours) (`sig_id`). 5000 unique drugs in total, with ~6 records each (no column that links the records).
- `df_tts`: 206 binary target mechanisms for the 23,814 training drugs.
- `df_ttn`: 402 additional unscored targets for the 23,814 training drugs for model development.

In [33]:
df, df_ttn, df_tts = get_base_datasets()
print(
    df.query('flag=="train"').shape,
    df.query('flag=="test"').shape,
    df_ttn.shape,
    df_tts.shape,
)

(23814, 877) (3982, 877) (23814, 403) (23814, 207)


In [34]:
# https://www.kaggle.com/pavelvpster/moa-keras-optuna-rfecv
# 500 + 3 + 2
selected_features = [
    "sig_id",
    "flag",
    "cp_type",
    "cp_time",
    "cp_dose",
    "g-0",
    "g-1",
    "g-2",
    "g-3",
    "g-4",
    "g-5",
    "g-6",
    "g-7",
    "g-8",
    "g-9",
    "g-10",
    "g-11",
    "g-12",
    "g-13",
    "g-14",
    "g-15",
    "g-16",
    "g-17",
    "g-18",
    "g-19",
    "g-20",
    "g-21",
    "g-22",
    "g-23",
    "g-24",
    "g-25",
    "g-26",
    "g-27",
    "g-28",
    "g-29",
    "g-30",
    "g-31",
    "g-32",
    "g-33",
    "g-34",
    "g-35",
    "g-36",
    "g-37",
    "g-38",
    "g-39",
    "g-40",
    "g-41",
    "g-42",
    "g-43",
    "g-44",
    "g-45",
    "g-46",
    "g-47",
    "g-48",
    "g-49",
    "g-50",
    "g-51",
    "g-52",
    "g-53",
    "g-54",
    "g-55",
    "g-56",
    "g-57",
    "g-58",
    "g-59",
    "g-60",
    "g-61",
    "g-62",
    "g-63",
    "g-64",
    "g-65",
    "g-66",
    "g-67",
    "g-68",
    "g-69",
    "g-70",
    "g-71",
    "g-72",
    "g-73",
    "g-74",
    "g-75",
    "g-76",
    "g-77",
    "g-78",
    "g-79",
    "g-80",
    "g-81",
    "g-82",
    "g-83",
    "g-84",
    "g-85",
    "g-86",
    "g-87",
    "g-88",
    "g-89",
    "g-90",
    "g-91",
    "g-92",
    "g-93",
    "g-94",
    "g-95",
    "g-96",
    "g-97",
    "g-98",
    "g-99",
    "g-100",
    "g-101",
    "g-102",
    "g-103",
    "g-104",
    "g-105",
    "g-106",
    "g-107",
    "g-108",
    "g-109",
    "g-110",
    "g-111",
    "g-112",
    "g-113",
    "g-114",
    "g-115",
    "g-116",
    "g-117",
    "g-118",
    "g-121",
    "g-122",
    "g-123",
    "g-125",
    "g-126",
    "g-127",
    "g-128",
    "g-129",
    "g-130",
    "g-131",
    "g-132",
    "g-133",
    "g-139",
    "g-144",
    "g-149",
    "g-152",
    "g-153",
    "g-154",
    "g-155",
    "g-158",
    "g-161",
    "g-162",
    "g-165",
    "g-166",
    "g-167",
    "g-168",
    "g-169",
    "g-170",
    "g-171",
    "g-172",
    "g-173",
    "g-174",
    "g-175",
    "g-176",
    "g-177",
    "g-178",
    "g-179",
    "g-180",
    "g-182",
    "g-185",
    "g-188",
    "g-189",
    "g-191",
    "g-192",
    "g-193",
    "g-194",
    "g-195",
    "g-196",
    "g-197",
    "g-198",
    "g-199",
    "g-200",
    "g-201",
    "g-202",
    "g-203",
    "g-204",
    "g-209",
    "g-211",
    "g-213",
    "g-214",
    "g-217",
    "g-220",
    "g-221",
    "g-222",
    "g-223",
    "g-225",
    "g-226",
    "g-229",
    "g-233",
    "g-239",
    "g-240",
    "g-243",
    "g-244",
    "g-246",
    "g-247",
    "g-250",
    "g-254",
    "g-255",
    "g-256",
    "g-257",
    "g-259",
    "g-261",
    "g-262",
    "g-265",
    "g-269",
    "g-270",
    "g-271",
    "g-273",
    "g-279",
    "g-282",
    "g-284",
    "g-286",
    "g-289",
    "g-292",
    "g-294",
    "g-295",
    "g-298",
    "g-299",
    "g-300",
    "g-303",
    "g-306",
    "g-310",
    "g-311",
    "g-312",
    "g-314",
    "g-316",
    "g-317",
    "g-318",
    "g-320",
    "g-321",
    "g-323",
    "g-326",
    "g-327",
    "g-328",
    "g-332",
    "g-333",
    "g-335",
    "g-339",
    "g-340",
    "g-341",
    "g-346",
    "g-347",
    "g-351",
    "g-353",
    "g-355",
    "g-357",
    "g-360",
    "g-364",
    "g-365",
    "g-370",
    "g-375",
    "g-383",
    "g-385",
    "g-388",
    "g-389",
    "g-390",
    "g-391",
    "g-392",
    "g-399",
    "g-401",
    "g-402",
    "g-403",
    "g-404",
    "g-409",
    "g-417",
    "g-423",
    "g-424",
    "g-425",
    "g-426",
    "g-428",
    "g-429",
    "g-430",
    "g-431",
    "g-433",
    "g-434",
    "g-438",
    "g-439",
    "g-440",
    "g-441",
    "g-443",
    "g-444",
    "g-445",
    "g-446",
    "g-447",
    "g-448",
    "g-449",
    "g-450",
    "g-451",
    "g-452",
    "g-453",
    "g-454",
    "g-455",
    "g-456",
    "g-460",
    "g-461",
    "g-463",
    "g-465",
    "g-467",
    "g-470",
    "g-473",
    "g-474",
    "g-475",
    "g-476",
    "g-481",
    "g-483",
    "g-485",
    "g-486",
    "g-489",
    "g-490",
    "g-495",
    "g-500",
    "g-501",
    "g-502",
    "g-504",
    "g-509",
    "g-513",
    "g-516",
    "g-517",
    "g-518",
    "g-527",
    "g-531",
    "g-535",
    "g-540",
    "g-542",
    "g-543",
    "g-544",
    "g-547",
    "g-548",
    "g-549",
    "g-552",
    "g-556",
    "g-558",
    "g-562",
    "g-566",
    "g-571",
    "g-572",
    "g-573",
    "g-574",
    "g-578",
    "g-584",
    "g-585",
    "g-590",
    "g-592",
    "g-595",
    "g-596",
    "g-600",
    "g-602",
    "g-603",
    "g-605",
    "g-606",
    "g-611",
    "g-612",
    "g-615",
    "g-617",
    "g-618",
    "g-619",
    "g-620",
    "g-623",
    "g-624",
    "g-625",
    "g-626",
    "g-627",
    "g-628",
    "g-629",
    "g-630",
    "g-631",
    "g-632",
    "g-633",
    "g-634",
    "g-635",
    "g-636",
    "g-637",
    "g-638",
    "g-639",
    "g-640",
    "g-643",
    "g-645",
    "g-646",
    "g-648",
    "g-651",
    "g-654",
    "g-657",
    "g-658",
    "g-659",
    "g-660",
    "g-661",
    "g-662",
    "g-663",
    "g-664",
    "g-665",
    "g-666",
    "g-668",
    "g-670",
    "g-672",
    "g-673",
    "g-674",
    "g-675",
    "g-677",
    "g-678",
    "g-679",
    "g-685",
    "g-689",
    "g-695",
    "g-697",
    "g-698",
    "g-699",
    "g-701",
    "g-702",
    "g-703",
    "g-704",
    "g-707",
    "g-709",
    "g-710",
    "g-712",
    "g-713",
    "g-714",
    "g-715",
    "g-721",
    "g-722",
    "g-724",
    "g-725",
    "g-726",
    "g-734",
    "g-736",
    "g-737",
    "g-740",
    "g-742",
    "g-743",
    "g-745",
    "g-747",
    "g-750",
    "g-751",
    "g-752",
    "g-753",
    "g-756",
    "g-757",
    "g-758",
    "g-759",
    "g-760",
    "g-761",
    "g-762",
    "g-763",
    "g-764",
    "g-766",
    "g-767",
    "g-768",
    "g-769",
    "g-770",
    "g-771",
    "c-0",
    "c-1",
    "c-2",
    "c-3",
    "c-4",
    "c-5",
    "c-11",
    "c-12",
    "c-14",
    "c-19",
    "c-20",
    "c-21",
    "c-22",
    "c-23",
    "c-25",
    "c-27",
    "c-32",
    "c-35",
    "c-36",
    "c-37",
    "c-40",
    "c-47",
    "c-54",
    "c-55",
    "c-57",
    "c-58",
    "c-61",
    "c-62",
    "c-63",
    "c-65",
    "c-66",
    "c-71",
    "c-74",
    "c-75",
    "c-77",
    "c-78",
    "c-79",
    "c-80",
    "c-81",
    "c-82",
    "c-83",
    "c-84",
    "c-85",
    "c-86",
    "c-87",
    "c-88",
    "c-89",
    "c-90",
    "c-91",
    "c-92",
    "c-93",
    "c-97",
]

In [35]:
df = df[selected_features]

In [36]:
df = pd.concat([df, pd.get_dummies(df["cp_type"], prefix="cp_type")], axis=1)
df = pd.concat([df, pd.get_dummies(df["cp_time"], prefix="cp_time")], axis=1)
df = pd.concat([df, pd.get_dummies(df["cp_dose"], prefix="cp_dose")], axis=1)
df = df.drop(["cp_type", "cp_time", "cp_dose"], axis=1)

## Train Model

In [46]:
X = df.query('flag=="train"').drop(["sig_id", "flag"], axis=1)
X_test = df.query('flag=="test"').drop(["sig_id", "flag"], axis=1)
y = df_tts.drop("sig_id", axis=1)

In [47]:
X.head()

,g-0,g-1,g-2,g-3,g-4,g-5,g-6,g-7,g-8,g-9,...,c-92,c-93,c-97,cp_type_ctl_vehicle,cp_type_trt_cp,cp_time_24,cp_time_48,cp_time_72,cp_dose_D1,cp_dose_D2
0,1.0620,0.5577,-0.2479,-0.6208,-0.1944,-1.0120,-1.0220,-0.0326,0.5548,-0.0921,...,0.8076,0.5523,0.2139,0,1,1,0,0,1,0
1,0.0743,0.4087,0.2991,0.0604,1.0190,0.5207,0.2341,0.3372,-0.4047,0.8507,...,0.4708,0.0230,0.1241,0,1,0,0,1,1,0
2,0.6280,0.5817,1.5540,-0.0764,-0.0323,1.2390,0.1715,0.2155,0.0065,1.2300,...,0.6103,0.0223,-0.2187,0,1,0,1,0,1,0
3,-0.5138,-0.2491,-0.2656,0.5288,4.0620,-0.8095,-1.9590,0.1792,-0.1321,-1.0600,...,-5.6300,-1.3780,-0.8784,0,1,0,1,0,1,0
4,-0.3254,-0.4009,0.9700,0.6919,1.4180,-0.8244,-0.2800,-0.1498,-0.8789,0.8630,...,0.6670,1.0690,0.2885,0,1,0,0,1,0,1


In [48]:
def fit_predict(n_splits, params, X, y, X_test):
    oof = np.zeros(X.shape[0])
    y_preds = []

    cv = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)
    for train_idx, valid_idx in cv.split(X, y):

        X_train = X.iloc[train_idx]
        y_train = y.iloc[train_idx]
        X_valid = X.iloc[valid_idx]
        y_valid = y.iloc[valid_idx]

        lgb_train = lgb.Dataset(
            data=X_train.astype("float32"), label=y_train.astype("float32")
        )
        lgb_valid = lgb.Dataset(
            data=X_valid.astype("float32"), label=y_valid.astype("float32")
        )

        estimator = lgb.train(
            params,
            lgb_train,
            10000,
            valid_sets=lgb_valid,
            early_stopping_rounds=25,
            verbose_eval=0,
        )

        oof_part = estimator.predict(X_valid, num_iteration=estimator.best_iteration)
        oof[valid_idx] = oof_part

        if X_test is not None:
            y_part = estimator.predict(X_test, num_iteration=estimator.best_iteration)
            y_preds.append(y_part)

    score = log_loss(y, oof)
    print(f"LogLoss Score: {score:.6f}")

    y_pred = np.mean(y_preds, axis=0)

    return y_pred, oof, score

In [49]:
%%time
start = time()

columns = [
    "glutamate_receptor_antagonist",
    "dna_inhibitor",
    "serotonin_receptor_antagonist",
    "dopamine_receptor_antagonist",
    "cyclooxygenase_inhibitor",
]  # selected columns against which we determine optimal hyperparameters


def objective(trial):
    params = {
        "objective": "binary",
        "metric": "binary_logloss",
        "boosting_type": "gbdt",
        "boost_from_average": True,
        "num_threads": -1,
        "random_state": 42,
        "num_leaves": trial.suggest_int("num_leaves", 10, 1000),
        "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 10, 200),
        "min_child_weight": trial.suggest_loguniform("min_child_weight", 0.001, 0.1),
        "max_depth": trial.suggest_int("max_depth", 1, 100),
        "bagging_fraction": trial.suggest_loguniform("bagging_fraction", 0.5, 0.99),
        "feature_fraction": trial.suggest_loguniform("feature_fraction", 0.5, 0.99),
        "lambda_l1": trial.suggest_loguniform("lambda_l1", 0.1, 2),
        "lambda_l2": trial.suggest_loguniform("lambda_l2", 0.1, 2),
    }

    scores = []
    for column in columns:
        print(f"Column: {column} - elapsed time: {(time()-start)/60:.1f} mins")
        _, _, score = fit_predict(3, params, X, y[column], None)
        scores.append(score)

    return np.mean(scores)


study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=100)

[I 2020-11-26 22:50:01,819] A new study created in memory with name: no-name-5667c9a1-abca-4429-bdd7-be741ca90ba2


Column: glutamate_receptor_antagonist - elapsed time: 0.0 mins
LogLoss Score: 0.079117
Column: dna_inhibitor - elapsed time: 0.1 mins


/opt/anaconda3/envs/moa/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3372: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/opt/anaconda3/envs/moa/lib/python3.8/site-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


LogLoss Score: 0.082303
Column: serotonin_receptor_antagonist - elapsed time: 0.3 mins


/opt/anaconda3/envs/moa/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3372: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/opt/anaconda3/envs/moa/lib/python3.8/site-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


...

LogLoss Score: 0.084284
Column: dopamine_receptor_antagonist - elapsed time: 40.3 mins


/opt/anaconda3/envs/moa/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3372: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/opt/anaconda3/envs/moa/lib/python3.8/site-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


LogLoss Score: 0.087185
Column: cyclooxygenase_inhibitor - elapsed time: 40.5 mins


/opt/anaconda3/envs/moa/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3372: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/opt/anaconda3/envs/moa/lib/python3.8/site-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/opt/anaconda3/envs/moa/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3372: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/opt/anaconda3/envs/moa/lib/python3.8/site-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
[I 2020-11-26 23:30:39,827] Trial 99 finished with value: 0.08418490607980374 and parameters: {'num_leaves': 923, 'min_data_in_leaf': 106, 'min_child_weight': 0.010683951047929342, 'max_depth': 18, 'bagging_fraction': 0.7110162510323492, 'feature_fraction': 0.7137849027039065, 'lambda_l1': 0.133354546

LogLoss Score: 0.089090
CPU times: user 8h 24min 26s, sys: 46min 49s, total: 9h 11min 15s
Wall time: 40min 38s


In [50]:
study.best_trial

FrozenTrial(number=20, value=0.0815351558993065, datetime_start=datetime.datetime(2020, 11, 26, 22, 58, 54, 547490), datetime_complete=datetime.datetime(2020, 11, 26, 22, 59, 12, 162541), params={'num_leaves': 975, 'min_data_in_leaf': 138, 'min_child_weight': 0.09896842978274249, 'max_depth': 2, 'bagging_fraction': 0.9725251479343076, 'feature_fraction': 0.5015261687098113, 'lambda_l1': 0.1713619024245054, 'lambda_l2': 0.20739273935313363}, distributions={'num_leaves': IntUniformDistribution(high=1000, low=10, step=1), 'min_data_in_leaf': IntUniformDistribution(high=200, low=10, step=1), 'min_child_weight': LogUniformDistribution(high=0.1, low=0.001), 'max_depth': IntUniformDistribution(high=100, low=1, step=1), 'bagging_fraction': LogUniformDistribution(high=0.99, low=0.5), 'feature_fraction': LogUniformDistribution(high=0.99, low=0.5), 'lambda_l1': LogUniformDistribution(high=2, low=0.1), 'lambda_l2': LogUniformDistribution(high=2, low=0.1)}, user_attrs={}, system_attrs={}, intermedi

In [51]:
study.best_trial.params

{'num_leaves': 975,
 'min_data_in_leaf': 138,
 'min_child_weight': 0.09896842978274249,
 'max_depth': 2,
 'bagging_fraction': 0.9725251479343076,
 'feature_fraction': 0.5015261687098113,
 'lambda_l1': 0.1713619024245054,
 'lambda_l2': 0.20739273935313363}